In [16]:
import os
import csv
import random
import json
import operator
from nltk.metrics.distance import masi_distance
from cluster import HierarchicalClustering

CSV_FILE = os.path.join('connections.csv')

OUT_FILE = 'viz/d3-data.json'


DISTANCE_THRESHOLD = 0.67
DISTANCE = masi_distance


SAMPLE_SIZE = 500
def cluster_contacts_by_title(csv_file):


    transforms = [
        ('Sr.', 'Senior'),
        ('Sr', 'Senior'),
        ('Jr.', 'Junior'),
        ('Jr','Junior'),
        ('CEO','Chief Executive Officer'),
        ('COO','Chief Operating Officer'),
        ('CTO','Chief Technology Officer'),
        ('CFO','Chief Finance Officer'),
        ('VP','Vice President'),
        
        ]

    tokenize_along = ['/','&','and']

    csvread = csv.DictReader(open(csv_file), delimiter=',', quotechar='"')
    connection = [row for row in csvread]
    titles=[]
   # print connection
    for i, _ in enumerate(connection):
        if connection[i]['Job Title'] == '':
            connection[i]['Job Titles'] = ['']
            continue
        indiv_title=[connection[i]['Job Title']]
        #print indiv_title
        for t in indiv_title:
            
            for seperator in tokenize_along:
                
                if t.find(seperator) >= 0:

                    indiv_title.remove(t)
                    t=t.strip()
                    temp=[]
                    if t!= '':
                        for title in t.split(seperator):
                            temp.append(title)

                        indiv_title.extend(temp)
        #print indiv_title
        for transform in transforms:
            indiv_title = [title.replace(*transform) for title in indiv_title]
        connection[i]['Job Titles'] = indiv_title
        titles.extend(indiv_title)
   # print len(titles)
    titles=list(set(titles))
    #print len(titles)
   
    #for t in titles:
     #   print t

    
   
    def score(title1, title2): 
        return DISTANCE(set(title1.split()), set(title2.split()))
    
    cluster={}
    titles2=titles
    for title1 in titles:
        
        cluster[title1]=[]
        for title2 in titles2:
            if title2 in cluster[title1]   or cluster.has_key(title2) and title1 in cluster[title2]:
                continue
            distance = DISTANCE(set(title1.split()), set(title2.split()))
            if distance < DISTANCE_THRESHOLD:
                cluster[title1].append(title2)
                titles2.remove(title2)
    cluster = [cluster[title] for title in cluster if len(cluster[title]) > 1]
   # print cluster
        
    contact_clustered={}
    for cl in cluster:
        contact_clustered[tuple(cl)]=[]
        for conn in connection:
            for t in conn['Job Titles']:
                if t in cl:
                     contact_clustered[tuple(cl)].append('%s %s'
                            % (conn['First Name'], conn['Last Name']))
    #print contact_clustered
    return contact_clustered
    



def display(clustered_contacts):
        for titles in clustered_contacts:
            common_titles_heading = 'Headings : ' + ', '.join(titles)
            print common_titles_heading
            ll=[]
            d=dict()
            for term_sep in titles:
                sp_sep_ti=term_sep.split()
                for sl in sp_sep_ti:
                    if sl in d:
                        d[sl]=d[sl]+1
                    else:
                        d[sl]=1;
            for key in d:
                if (d[key] * 2) >= (len(d)):
                    print "\t  ",key
            print '-' * len(common_titles_heading)
            for t in clustered_contacts[titles]:
                print "\t  ",t;
            print'\n'
            
def write_d3_json_output(clustered_contacts):
    
    json_output = {'name' : 'My LinkedIn', 'children' : []}

    for titles in clustered_contacts:

        descriptive_terms = set(titles[0].split())
        #print descriptive_terms
        for title in titles:
            descriptive_terms.intersection_update(set(title.split()))
        ll=[]
        d=dict()
        
        for term_sep in titles:
         #   print term_sep
            sp_sep_ti=term_sep.split()
        #    print sp_sep_ti
            for sl in sp_sep_ti:
                if sl in d:
                    d[sl]=d[sl]+1
                else:
                    d[sl]=1;
            #ll.append(sl)
        print d
        a=[]
        for key in d:
            #  print d[key]
            if (d[key] ) >= (len(titles)/2):
  #              print key
                a.append(key)
                
        print a
        print descriptive_terms

        json_output['children'].append({'name' : ', '.join(a), 
                                    'children' : [ {'name' : c.decode('utf-8', 'replace')} for c in clustered_contacts[titles] ] } )
    
        f = open(OUT_FILE, 'w')
        f.write(json.dumps(json_output, indent=1))
        f.close()  
clustered_contacts = cluster_contacts_by_title(CSV_FILE)
display(clustered_contacts)
write_d3_json_output(clustered_contacts)

Headings : Product Manager, Manager
	   Product
	   Manager
-----------------------------------
	   shaloo singh
	   Nasiruddin Ahmad


Headings :  Programmer, Programmer Analyst
	   Programmer
	   Analyst
------------------------------------------
	   JP Godara
	   Ramanan Ramesh


Headings : Professor, Assistant Professor
	   Assistant
	   Professor
-----------------------------------------
	   Monark Bag
	   sanjai singh
	   Samudrala Kumar


Headings : Student , ECE Student, Student Ambassador, Student
	   Student
-------------------------------------------------------------
	   Manu Saxena
	   Abhimanyu Singh
	   Harshit Gupta
	   Monica Singh
	   ravinder naik azmeera
	   Jyoti Singh
	   Vishal Srivastava
	   Aman Agarwal
	   Jai Mishra
	   Upendra gupta
	   Shivlal Dhundhwal
	   Amarnath kumar
	   Nishtha Rai
	   Vikhyat Tandon
	   Shubham Tyagi
	   siyaram malav
	   manish kumar
	   Vishnu Ks
	   Sumit Kumar
	   Amit Kumar Singh
	   Shailendra Pratap Singh
	   Ashutosh Gupta
	 

In [25]:
import os
import csv
import random
import json
import operator
from nltk.metrics.distance import jaccard_distance
from cluster import HierarchicalClustering

CSV_FILE = os.path.join('connections.csv')

OUT_FILE = 'viz/d4-data.json'


DISTANCE_THRESHOLD = 0.59
DISTANCE = jaccard_distance


SAMPLE_SIZE = 500
def cluster_contacts_by_title(csv_file):


    transforms = [
        ('Sr.', 'Senior'),
        ('Sr', 'Senior'),
        ('Jr.', 'Junior'),
        ('Jr','Junior'),
        ('CEO','Chief Executive Officer'),
        ('COO','Chief Operating Officer'),
        ('CTO','Chief Technology Officer'),
        ('CFO','Chief Finance Officer'),
        ('VP','Vice President'),
        
        ]

    tokenize_along = ['/','&','and']

    csvread = csv.DictReader(open(csv_file), delimiter=',', quotechar='"')
    connection = [row for row in csvread]
    titles=[]
   # print connection
    for i, _ in enumerate(connection):
        if connection[i]['Job Title'] == '':
            connection[i]['Job Titles'] = ['']
            continue
        print connection[i]['Job Title'],connection[i]['First Name'],connection[i]['Last Name']
        indiv_title=[connection[i]['Job Title']]
        #print indiv_title
        for t in indiv_title:
            
            for seperator in tokenize_along:
                
                if t.find(seperator) >= 0:

                    indiv_title.remove(t)
                    t=t.strip()
                    temp=[]
                    if t!= '':
                        for title in t.split(seperator):
                            temp.append(title)

                        indiv_title.extend(temp)
        #print indiv_title
        for transform in transforms:
            indiv_title = [title.replace(*transform) for title in indiv_title]
        connection[i]['Job Titles'] = indiv_title
        titles.extend(indiv_title)
   # print len(titles)

    titles=list(set(titles))
    #print len(titles)
   
    #for t in titles:
     #   print t

    
   
    def score(title1, title2): 
        return DISTANCE(set(title1.split()), set(title2.split()))
    
    cluster={}
    titles2=titles
    for title1 in titles:
        
        cluster[title1]=[]
        for title2 in titles2:
            if title2 in cluster[title1]   or cluster.has_key(title2) and title1 in cluster[title2]:
                continue
            distance = DISTANCE(set(title1.split()), set(title2.split()))
            if distance < DISTANCE_THRESHOLD:
                cluster[title1].append(title2)
                titles2.remove(title2)
    cluster = [cluster[title] for title in cluster if len(cluster[title]) > 1]
   # print cluster
        
    contact_clustered={}
    for cl in cluster:
        contact_clustered[tuple(cl)]=[]
        for conn in connection:
            for t in conn['Job Titles']:
                if t in cl:
                     contact_clustered[tuple(cl)].append('%s %s'
                            % (conn['First Name'], conn['Last Name']))
    #print contact_clustered
    return contact_clustered
    



def display(clustered_contacts):
        for titles in clustered_contacts:
            common_titles_heading = 'Headings : ' + ', '.join(titles)
            print common_titles_heading
            ll=[]
            d=dict()
            for term_sep in titles:
                sp_sep_ti=term_sep.split()
                for sl in sp_sep_ti:
                    if sl in d:
                        d[sl]=d[sl]+1
                    else:
                        d[sl]=1;
            for key in d:
                if (d[key] * 2) >= (len(d)):
                    print "\t  ",key
            print '-' * len(common_titles_heading)
            for t in clustered_contacts[titles]:
                print "\t  ",t;
            print'\n'
            
def write_d3_json_output(clustered_contacts):
    
    json_output = {'name' : 'My LinkedIn', 'children' : []}

    for titles in clustered_contacts:

        descriptive_terms = set(titles[0].split())
        #print descriptive_terms
        for title in titles:
            descriptive_terms.intersection_update(set(title.split()))
        ll=[]
        d=dict()
        
        for term_sep in titles:
         #   print term_sep
            sp_sep_ti=term_sep.split()
        #    print sp_sep_ti
            for sl in sp_sep_ti:
                if sl in d:
                    d[sl]=d[sl]+1
                else:
                    d[sl]=1;
            #ll.append(sl)
        print d
        a=[]
        for key in d:
            #  print d[key]
            if (d[key] ) >= (len(titles)/2):
  #              print key
                a.append(key)
                
        print a
        print descriptive_terms

        json_output['children'].append({'name' : ', '.join(a), 
                                    'children' : [ {'name' : c.decode('utf-8', 'replace')} for c in clustered_contacts[titles] ] } )
    
        f = open(OUT_FILE, 'w')
        f.write(json.dumps(json_output, indent=1))
        f.close()  
clustered_contacts = cluster_contacts_by_title(CSV_FILE)
display(clustered_contacts)
write_d3_json_output(clustered_contacts)

Student Manu Saxena
Assistant Professor Monark Bag
Assistant Developer Lakshya Goel
Student Abhimanyu Singh
Student Ambassador Harshit Gupta
Software Developer Sushil Kumar
Head, Technical Affairs-Software Gitanshu Behal
Student Monica Singh
Student ravinder naik azmeera
Student Jyoti Singh
Sofware Engineer koganti rakesh
Software Engineer Aviral Sahai
Software Engineer sarthak mall
Software Engineer Udit Agarwal
Student Vishal Srivastava
Student Aman Agarwal
Software Engineer deepak irukula
Software Engineer Sunil Kumar
Software Engineer Tejaswi Reddy
Human Resources Aparna Khanna
Student Jai Mishra
Student Upendra gupta
Firefox Student Ambassador Charul Agrawal
Placement Coordinator Vijendra Singh Aswal
Filmmaker/Writer Yeswanth Mocharla
Data Developer Prabhat Kumar
Software Developmer Akshat Sapra
Student Shivlal Dhundhwal
Software Engineer Rahul Sharma
Software Engineer Harsh Lalwani
Software Developer seshank koganti
Software Engineer Sapan Diwakar
Co-founder Harsha Suryanarayana


In [20]:
import os
import csv
import random
import json
import operator
from nltk.metrics.distance import edit_distance
from cluster import HierarchicalClustering

CSV_FILE = os.path.join('connections.csv')

OUT_FILE = 'viz/d5-data.json'


DISTANCE_THRESHOLD = 6
DISTANCE = edit_distance


SAMPLE_SIZE = 500
def cluster_contacts_by_title(csv_file):


    transforms = [
        ('Sr.', 'Senior'),
        ('Sr', 'Senior'),
        ('Jr.', 'Junior'),
        ('Jr','Junior'),
        ('CEO','Chief Executive Officer'),
        ('COO','Chief Operating Officer'),
        ('CTO','Chief Technology Officer'),
        ('CFO','Chief Finance Officer'),
        ('VP','Vice President'),
        
        ]

    tokenize_along = ['/','&','and']

    csvread = csv.DictReader(open(csv_file), delimiter=',', quotechar='"')
    connection = [row for row in csvread]
    titles=[]
   # print connection
    for i, _ in enumerate(connection):
        if connection[i]['Job Title'] == '':
            connection[i]['Job Titles'] = ['']
            continue
        indiv_title=[connection[i]['Job Title']]
#  print indiv_title
        for t in indiv_title:
            
            for seperator in tokenize_along:
                
                if t.find(seperator) >= 0:

                    indiv_title.remove(t)
                    t=t.strip()
                    temp=[]
                    if t!= '':
                        for title in t.split(seperator):
                            temp.append(title)

                        indiv_title.extend(temp)
        
        for transform in transforms:
            indiv_title = [title.replace(*transform) for title in indiv_title]
        connection[i]['Job Titles'] = indiv_title
        titles.extend(indiv_title)

    titles=list(set(titles))
   

    
    
   
    def score(title1, title2): 
        return DISTANCE(title1, title2)
    
    cluster={}
    titles2=titles
    for title1 in titles:
        cluster[title1]=[]
        for title2 in titles2:
            if title2 in cluster[title1]   or cluster.has_key(title2) and title1 in cluster[title2]:
                continue
            distance = DISTANCE(title1, title2)
            if distance < DISTANCE_THRESHOLD:
                cluster[title1].append(title2)
                titles2.remove
    cluster = [cluster[title] for title in cluster if len(cluster[title]) > 1]
    #print cluster
        
    contact_clustered={}
    for cl in cluster:
        contact_clustered[tuple(cl)]=[]
        for conn in connection:
            for t in conn['Job Titles']:
                if t in cl:
                     contact_clustered[tuple(cl)].append('%s %s'
                            % (conn['First Name'], conn['Last Name']))
    #print contact_clustered
    return contact_clustered
    



def display(clustered_contacts):
        for titles in clustered_contacts:
            common_titles_heading = 'Headings : ' + ', '.join(titles)
            print common_titles_heading
            ll=[]
            d=dict()
            for term_sep in titles:
                sp_sep_ti=term_sep.split()
                for sl in sp_sep_ti:
                    if sl in d:
                        d[sl]=d[sl]+1
                    else:
                        d[sl]=1;
            for key in d:
                if (d[key] * 2) >= (len(d)):
                    print "\t  ",key
            print '-' * len(common_titles_heading)
            for t in clustered_contacts[titles]:
                print "\t  ",t;
            print'\n'
            
def write_d3_json_output(clustered_contacts):
    
    json_output = {'name' : 'My LinkedIn', 'children' : []}

    for titles in clustered_contacts:

        descriptive_terms = set(titles[0].split())
        #print descriptive_terms
        for title in titles:
            descriptive_terms.intersection_update(set(title.split()))
        ll=[]
        d=dict()
        
        for term_sep in titles:
         #   print term_sep
            sp_sep_ti=term_sep.split()
        #    print sp_sep_ti
            for sl in sp_sep_ti:
                if sl in d:
                    d[sl]=d[sl]+1
                else:
                    d[sl]=1;
            #ll.append(sl)
        print d
        a=[]
        for key in d:
            #  print d[key]
            if (d[key] ) >= (len(titles)/2):
  #              print key
                a.append(key)
                
        print a
        print descriptive_terms

        json_output['children'].append({'name' : ', '.join(a), 
                                    'children' : [ {'name' : c.decode('utf-8', 'replace')} for c in clustered_contacts[titles] ] } )
    
        f = open(OUT_FILE, 'w')
        f.write(json.dumps(json_output, indent=1))
        f.close()  
clustered_contacts = cluster_contacts_by_title(CSV_FILE)
display(clustered_contacts)
write_d3_json_output(clustered_contacts)

Headings : Systems Engineer, system engineer
--------------------------------------------
	   Rohit Wadhwa
	   manoj verma


Headings : Assistant Professor, Associate Professor
	   Professor
---------------------------------------------------
	   Monark Bag
	   sanjai singh
	   Jag Rawat


Headings : Web Designer, Web Designer 
	   Web
	   Designer
--------------------------------------
	   Deepak Gupta
	   JP Godara


Headings : Service Engineer, Sofware Engineer
	   Engineer
---------------------------------------------
	   koganti rakesh
	   praveen kumar


Headings : Consultant, IT Consultant
	   Consultant
	   IT
------------------------------------
	   Pradeep Mishra
	   raja agarwal
	   ASHISH KUMAR SAHU


Headings : Data Developer, Developer
	   Data
	   Developer
------------------------------------
	   Prabhat Kumar
	   Vedavyas Chigurupati
	   Suyash Agarwal


Headings : Founder , Student , Co-founder, Student
	   Student
--------------------------------------------------
	 